## Baseline Model

This workbook implements model that makes predictions based on word counts, using CountVectorizer

In [ ]:
# General libraries.
import json
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import datetime

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn import mixture
from sklearn.model_selection import train_test_split

# SK-learn libraries for evaluation.
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

# NLTK APIs
from nltk import pos_tag, sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer


### Load Data

In [ ]:
datapath = 'data/fakeNewsDatasets_Perez-Rosas2018'

# Function to load news articles
def tabulate_data(dataset_name):
    """Create a Pandas dataframe out of input Perez-Rosas dataset files
    @param dataset_name: Name of the dataset (fakenews or celebrity)
    @returns Pandas dataframe with columns:
        dataset_name, news_type, news_category, news_headline, news_content
    """
    def remove_numbers(in_str):
        return re.sub(r'[0-9]+', '', in_str)

    result_data_list = []
    data_dir = datapath
    for news_type in ['fake', 'legit']:
        folder = '%s/%s/%s' % (data_dir, dataset_name, news_type)
        for fname in os.listdir(folder):
            result_data = {}
            result_data['dataset_name'] = dataset_name
            result_data['news_type'] = news_type
            if news_type == 'fake':
                result_data['is_fake'] = 1
            else:
                result_data['is_fake'] = 0
            if dataset_name == 'fakeNewsDataset':
                result_data['news_category'] = remove_numbers(fname.split('.')[0])
            result_data['file_name'] = fname
            filepath = os.path.join(folder, fname)
            with open(filepath, 'r', encoding="utf8") as f:
                file_data = f.read().split('\n')
                # Some articles don't have a headline, but only article body.
                if len(file_data) > 1:
                    news_content_data = ' '.join(file_data[2:])
                    result_data['news_headline'] = file_data[0]
                else:
                    news_content_data = file_data[0]
                    result_data['news_headline'] = ''
                result_data['news_content'] = news_content_data
                result_data['news_all'] = ' '.join(file_data[0:])
                result_data_list.append(result_data)
    df = pd.DataFrame(result_data_list)
    return df

In [ ]:
# Load fake news data
fakenews_df = tabulate_data('fakeNewsDataset')
fakenews_df.head()

In [ ]:
# Split data into training and test sets
newsVectors, newsVectors_test, classVector, classVector_test = \
    train_test_split(fakenews_df['news_all'],
                     fakenews_df['is_fake'], 
                     test_size = .2,
                     random_state = 1)

### Functions to Preprocess Text

In [ ]:
nltk.download('stopwords')

In [ ]:
# Removing stemming and lemmatization

wnl = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    return [wnl.lemmatize(word.lower(), pos=get_wordnet_pos(tag)) 
            for word, tag in pos_tag(word_tokenize(text))]

In [ ]:
# preprocessor

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', 'numseq', text)
    text = re.sub(r'[^a-zA-Z]+', ' ', text)
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text)
    
    filtered_text = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_text.append(w) 
            
    return filtered_text


### Vectorize Data

In [ ]:
vectorizer = CountVectorizer(analyzer=preprocess_text)
train_data = vectorizer.fit_transform(newsVectors)
test_data = vectorizer.transform(newsVectors_test)

print("Size of the vocabulary:", train_data.shape[1])

##### Multinomial Naive Bayes Model

In [ ]:
# Multinomial modeling
alphas = {'alpha': [0.01, 0.05, 0.1, 0.5, 0.8, 1, 1.5, 5, 10]}
multi_gs = GridSearchCV(estimator=MultinomialNB(), param_grid=alphas)
multi_gs.fit(train_data, classVector)

multi_clf_best = MultinomialNB(alpha=multi_gs.best_params_['alpha'])
multi_clf_best.fit(train_data, classVector)
multi_clf_best_predicted = multi_clf_best.predict(test_data)
fpr, tpr, _ = metrics.roc_curve(classVector_test, multi_clf_best_predicted)

print('----- Multinomial Naive Bayes -----')
print (multi_gs.best_params_)
print("Multinomial accuracy: {:2.5f}%".format(accuracy_score(multi_clf_best_predicted, classVector_test) * 100))
print("Multinomial accuracy (AUC): {:2.5f}%".format(metrics.auc(fpr,tpr) * 100))

print('Number of predictions:', len(multi_clf_best_predicted))
print('Number predicted as Fake News:',sum(multi_clf_best_predicted))
print('-----------------------------------')
print('--- Confusion Matrix ---')
print(confusion_matrix(classVector_test, multi_clf_best_predicted))

##### Logistic Regression Model

In [ ]:
# Logisitc modeling

log_clf = LogisticRegression()
C = {"C": [0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.5, 0.8, 1, 1.5, 5, 10]}
log_gs = GridSearchCV(log_clf,C)
log_gs.fit(train_data, classVector)

log_clf_best = LogisticRegression(C=log_gs.best_params_['C'])
log_clf_best.fit(train_data, classVector)
log_clf_best_predicted = log_clf_best.predict(test_data)
fpr, tpr, _ = metrics.roc_curve(classVector_test, log_clf_best_predicted)

print('----- Logistic Regression -----')
print (log_gs.best_params_)
print ("Logistic accuracy: {:2.5f}%".format(accuracy_score(log_clf_best_predicted, classVector_test) * 100))
print ("Logistic accuracy (AUC): {:2.5f}%".format(metrics.auc(fpr,tpr) * 100))

# print confusion matrix to identify mistakes
print('--- Confusion Matrix ---')
print (confusion_matrix(classVector_test, log_clf_best_predicted))
print('-----------------------------------')